In [ ]:
import findspark
findspark.init()

import pyspark
from pyspark.sql import *
# import pyspark.sql.functions as func

### Load Data

In [1]:
posts_df = spark.read.json("data/Posts.json")

NameError: name 'spark' is not defined